In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Set the seed
tf.random.set_seed(42)

# Preprocess data (get all of the pixel values between 1 and 0, also called scaling/normalization)
train_datagen = ImageDataGenerator(rescale=1./255)
valid_datagen = ImageDataGenerator(rescale=1./255)

# Setup the train and test directories
train_dir = "FRUIT-16K-PREPROCESSED/train/" # 80% of FRUIT-16K
test_dir = "FRUIT-16K-PREPROCESSED/test/" # 20% of FRUIT-16K

# Import data from directories and turn it into batches
train_data = train_datagen.flow_from_directory(train_dir,
                                               batch_size=32,
                                               target_size=(224, 224),
                                               class_mode="categorical",
                                               seed=42)

valid_data = valid_datagen.flow_from_directory(test_dir,
                                               batch_size=32,
                                               target_size=(224, 224),
                                               class_mode="categorical",
                                               seed=42)

Found 12800 images belonging to 16 classes.
Found 3200 images belonging to 16 classes.


In [2]:
checkpoint_filepath = './model_5_callbacks'
model_5_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

In [3]:
base_model = tf.keras.applications.xception.Xception(include_top=False, weights='imagenet')
base_model.trainable = False

inputs = tf.keras.layers.Input(shape=(224, 224, 3), name="input_layer")

x = base_model(inputs)
print(f"Shape after base_model: {x.shape}")

x = tf.keras.layers.GlobalAveragePooling2D(name="global_average_pooling_layer")(x)
print(f"After GlobalAveragePooling2D(): {x.shape}")

outputs = tf.keras.layers.Dense(16, activation="softmax", name="output_layer")(x)

model_5 = tf.keras.Model(inputs, outputs)

model_5.compile(loss="categorical_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

history_5 = model_5.fit(train_data,
                        epochs=5,
                        validation_data=valid_data,
                        validation_steps=len(valid_data),
                        callbacks=[model_5_checkpoint_callback])

Shape after base_model: (None, 7, 7, 2048)
After GlobalAveragePooling2D(): (None, 2048)
Epoch 1/5
400/400 [==============================] - 34s 71ms/step - loss: 0.3575 - accuracy: 0.9249 - val_loss: 0.1181 - val_accuracy: 0.9750
Epoch 2/5
400/400 [==============================] - 28s 70ms/step - loss: 0.0714 - accuracy: 0.9894 - val_loss: 0.0644 - val_accuracy: 0.9872
Epoch 3/5
400/400 [==============================] - 28s 69ms/step - loss: 0.0403 - accuracy: 0.9951 - val_loss: 0.0439 - val_accuracy: 0.9941
Epoch 4/5
400/400 [==============================] - 30s 74ms/step - loss: 0.0260 - accuracy: 0.9977 - val_loss: 0.0353 - val_accuracy: 0.9944
Epoch 5/5
400/400 [==============================] - 28s 69ms/step - loss: 0.0184 - accuracy: 0.9983 - val_loss: 0.0306 - val_accuracy: 0.9937
